# Language modelling

In [1]:
import warnings
warnings.filterwarnings("ignore")

### Download three Polish models from the Huggingface repository. 

In [2]:
from transformers import pipeline, set_seed

herbert = pipeline(task="fill-mask", model='allegro/herbert-base-cased')
roberta = pipeline("fill-mask", model="sdadas/polish-roberta-base-v2")
papuga = pipeline("text-generation", model="dkleczek/papuGaPT2")
set_seed(42)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


### Devise a method to test if the langage model understands Polish cases.

In [24]:
test_cases = {"nominative" : "Warszawa to największe <mask>.",
"genitive" : "W Polsce nie ma <mask>.",
"dative" : "Będąc w Muzeum Narodowym przyglądałem się <mask>.",
"accusative" : "Nad morzem podziwiałem <mask>.",
"instrumental" : "Nie zgadzam się z <mask>.",
"locative" : "Przed snem myślałem o <mask>.",
"vocative" : "Witaj, mój najlepszy <mask>."}

models = {"HerBERT" : herbert, "RoBERTa": roberta, "papuGaPT2": papuga}

for case, text in test_cases.items():
    print(f"\nTest for {case}:")
    for model_name, model in models.items():
        if model_name == "papuGaPT2":
            text = text.replace("<mask>", "").replace(".", "").strip()
            print(text)
        
        preds = model(text, top_k=3)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"Prediction: {pred['generated_text']}\n")
            else:
                print(f"Prediction: {pred['sequence']}, Score: {pred['score']:.4f}\n")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Test for nominative:

Predictions of HerBERT:
Prediction: Warszawa to największe miasto ., Score: 0.8104

Prediction: Warszawa to największe lotnisko ., Score: 0.0825

Prediction: Warszawa to największe centrum ., Score: 0.0265


Predictions of RoBERTa:
Prediction: Warszawa to największe miasto ., Score: 0.7344

Prediction: Warszawa to największe zło ., Score: 0.0098

Prediction: Warszawa to największe centrum ., Score: 0.0083

Warszawa to największe


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Warszawa to największe miasto województwa śląskiego, a także drugie pod względem wielkości miasto Polski. Położone w województwie śląskim, jest także jednym z najważniejszych miast w kraju.
Miasto jest położone w południowej części województwa śląskiego. W mieście znajduje się wiele zabytków, które warto


Test for genitive:

Predictions of HerBERT:
Prediction: W Polsce nie ma internetu ., Score: 0.0224

Prediction: W Polsce nie ma autostrad ., Score: 0.0157

Prediction: W Polsce nie ma kanalizacji ., Score: 0.0148


Predictions of RoBERTa:
Prediction: W Polsce nie ma ., Score: 0.0576

Prediction: W Polsce nie ma 3 ., Score: 0.0123

Prediction: W Polsce nie ma 5 ., Score: 0.0105

W Polsce nie ma


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


Predictions of papuGaPT2:
Prediction: Nad morzem podziwiałem widoki na morze, które jest bardzo czyste. W tym roku postanowiłem wybrać się na wycieczkę rowerową.
W tym roku wybrałem się na wycieczkę rowerową. W planach było zobaczenie Kołobrzegu, które jest piękne i ma bardzo dużo do zaoferowania.


Test for instrumental:

Predictions of HerBERT:
Prediction: Nie zgadzam się z tym ., Score: 0.3859

Prediction: Nie zgadzam się z nim ., Score: 0.0694

Prediction: Nie zgadzam się z Tobą ., Score: 0.0691


Predictions of RoBERTa:
Prediction: Nie zgadzam się z tym ., Score: 0.1140

Prediction: Nie zgadzam się z nim ., Score: 0.0500

Prediction: Nie zgadzam się z tobą ., Score: 0.0362

Nie zgadzam się z


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Nie zgadzam się z tym, że nie można być dobrym w każdej dziedzinie. To nie jest tak, że wszyscy jesteśmy doskonałymi w każdej dziedzinie, ale w każdym z nas są dobrzy i tacy, którzy są doskonałymi w każdej dziedzinie.
To jest


Test for locative:

Predictions of HerBERT:
Prediction: Przed snem myślałem o tym ., Score: 0.0917

Prediction: Przed snem myślałem o tobie ., Score: 0.0673

Prediction: Przed snem myślałem o pracy ., Score: 0.0358


Predictions of RoBERTa:
Prediction: Przed snem myślałem o tym ., Score: 0.0409

Prediction: Przed snem myślałem o ., Score: 0.0211

Prediction: Przed snem myślałem o nim ., Score: 0.0203

Przed snem myślałem o


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Przed snem myślałem o tym, jak bardzo chciałbym być w tym miejscu. Nie wiem, jak to się stało, ale w końcu się zdecydowałem, że to będzie moje miejsce. Nie mogłem się zdecydować. To było takie dziwne. Nie wiedziałem, jak to się


Test for vocative:

Predictions of HerBERT:
Prediction: Witaj , mój najlepszy przyjaciel ., Score: 0.5444

Prediction: Witaj , mój najlepszy znajomy ., Score: 0.0961

Prediction: Witaj , mój najlepszy kolega ., Score: 0.0133


Predictions of RoBERTa:
Prediction: Witaj, mój najlepszy ., Score: 0.1601

Prediction: Witaj, mój najlepszy przyjaciel ., Score: 0.0893

Prediction: Witaj, mój najlepszy 2 ., Score: 0.0599

Witaj, mój najlepszy

Predictions of papuGaPT2:
Prediction: Witaj, mój najlepszy przyjaciel, jest to mój nowy przyjaciel. Jestem bardzo szczęśliwy, że mogę go poznać. Mam nadzieję, że będzie mi miło, gdy będę mógł go poznać.
- Nie mogę się doczekać, aby go poznać, ponieważ jest to mój



Modele papuGaPT2 i HerBERT dobrze poradziły sobie z odmianą przez przypadki, miały jedynie problem z wołaczem. Najgorzej poradził siebie model RoBERTa, który często w ogóle nie uzupełniał maski lub uzupełniał ją cyframi. Wszystkie modele miały problem z wołaczem.

### Devise a method to test long-range relationships such as gende

In [ ]:
sentences = ["Wiktor oświadczył się Matyldzie, a potem się z nią <mask>.",
             "Olek przygotował sobie obiad, ale przyszła głodna Baśka i go <mask>.",
             "Monika długo przygotowywała się do konkursu, ale to Robert go <mask>.",
             "Ala znalazła bezdomnego kota i go <mask>."]

for text in sentences:
    for model_name, model in models.items():
        if model_name == "papuGaPT2":
            text = text.replace("<mask>", "").replace(".", "").strip()
        preds = model(text, top_k=1)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"Prediction: {pred['generated_text']}\n")
            else:
                print(f"Prediction: {pred['sequence']}, Score: {pred['score']:.4f}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of HerBERT:
Prediction: Wiktor oświadczył się Matyldzie , a potem się z nią ożenił ., Score: 0.7896


Predictions of RoBERTa:
Prediction: Wiktor oświadczył się Matyldzie, a potem się z nią ożenił ., Score: 0.4026



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Wiktor oświadczył się Matyldzie, a potem się z nią ożenił. W tym czasie w domu Matyldy, w którym mieszkała, było już dwóch braci.
W domu Matyldy, w którym mieszkała Matylda, było już dwóch


Predictions of HerBERT:
Prediction: Olek przygotował sobie obiad , ale przyszła głodna Baśka i go zabrała ., Score: 0.3941


Predictions of RoBERTa:
Prediction: Olek przygotował sobie obiad, ale przyszła głodna Baśka i go nie ., Score: 0.0568



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Olek przygotował sobie obiad, ale przyszła głodna Baśka i gotowała. Po obiedzie, gdy już wszyscy byli gotowi, przyszedł do niej Olek i powiedział:
- Olek, nie płacz, bo ja już nie dam rady. Ja już


Predictions of HerBERT:
Prediction: Monika długo przygotowywała się do konkursu , ale to Robert go wygrał ., Score: 0.8962


Predictions of RoBERTa:
Prediction: Monika długo przygotowywała się do konkursu, ale to Robert go wygrał ., Score: 0.5036



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Monika długo przygotowywała się do konkursu, ale to Robert go wygrał. W końcu to on wygrał.
- To był mój pierwszy konkurs, w którym brałam udział. Nie miałam pojęcia, że to jest takie trudne. Nie wiedziałam, że to


Predictions of HerBERT:
Prediction: Ala znalazła bezdomnego kota i go oddała ., Score: 0.2584


Predictions of RoBERTa:
Prediction: Ala znalazła bezdomnego kota i go ., Score: 0.0883


Predictions of papuGaPT2:
Prediction: Ala znalazła bezdomnego kota i go przygarnęła. Nie miała jednak pojęcia, że to nie jest jej kot.
- Nie, nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie, nie. Nie, nie



Modele dobrze poradziły sobie z dopasowywaniem rodzaju czasownika do podmiotu. Ponownie najgorzej wypadł model RoBERTa, który często nie uzupełniał maski. Modele papuGaPT2 i HerBERT stosowały poprawne formy, nawet gdy drugi czasownik wymagał zastosowania innego rodzaju niż pierwszy.

### Check if the model captures real-world knolwedge

In [39]:
konowledge_tests = ["Bitwa pod Grunwaldem była w roku <mask>.",
                    "Stolicą Węgier jest <mask>.",
                    "Głównym składnikiem powietrza jest <mask>."]
for text in konowledge_tests:
    for model_name, model in models.items():
        if model_name == "papuGaPT2":
            text = text.replace("<mask>", "").replace(".", "").strip()
        preds = model(text, top_k=1)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"Prediction: {pred['generated_text']}\n")
            else:
                print(f"Prediction: {pred['sequence']}, Score: {pred['score']:.4f}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of HerBERT:
Prediction: Bitwa pod Grunwaldem była w roku 1300 ., Score: 0.0712


Predictions of RoBERTa:
Prediction: Bitwa pod Grunwaldem była w roku 1500 ., Score: 0.1965



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Bitwa pod Grunwaldem była w roku 1410 wielkim zwycięstwem wojsk polsko-litewskich. W bitwie tej, która miała miejsce w roku 1410, brało udział około tysiąca rycerzy, którzy walczyli w obronie Rzeczypospolitej.
W roku 1410,


Predictions of HerBERT:
Prediction: Stolicą Węgier jest Ferenc ., Score: 0.3555


Predictions of RoBERTa:
Prediction: Stolicą Węgier jest Budapeszt ., Score: 0.9932



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Prediction: Stolicą Węgier jest Budapeszt. W stolicy Węgier znajduje się wiele zabytków, które warto zobaczyć. Jednym z nich jest Zamek Królewski, który jest jednym z najważniejszych zabytków w Europie.
W Budapeszcie znajduje się wiele muzeów, które warto zobaczyć.


Predictions of HerBERT:
Prediction: Głównym składnikiem powietrza jest woda ., Score: 0.4580


Predictions of RoBERTa:
Prediction: Głównym składnikiem powietrza jest woda ., Score: 0.1810


Predictions of papuGaPT2:
Prediction: Głównym składnikiem powietrza jest dwutlenek węgla, który jest głównym składnikiem atmosfery. W powietrzu znajduje się około 10% tlenu, a w wodzie około 10%.
W powietrzu znajduje się około 10% tlenu, a w wodzie około 10%. W powietrzu znajduje się około



Modele słabo poradziły sobie z testem wiedzy. Najlepiej wypadł model papuGaPT2, który poprawnie odpowiedział na dwa pytania. Model RoBERTa udzielił jednej poprawnej odpowiedzi. Najsłabiej wypadł model HerBERT, który nie uzupełnij poprawnie żdanego zdania.

### Check zero-shot learning capabilites of the models.

In [ ]:
reviews = ["Film był tak nudny, że zsnęłam w kinie.",
                   "Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny.",
                   "Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty.",
                   "Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem.",
                   "Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać."]
prompts = ["Zabarwienie emocjonalne wypowiedzi jest <mask>.",
           "Wypowiedź ma zdecydowanie wydźwięk <mask>.",
           "Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest <mask>."]

sentiment_tests = [(review, prompt) for review in reviews for prompt in prompts]

prompt = prompts[0]
print(f"\nPrompt: {prompt}")
for review in reviews:
    full_text = f"{review}\n{prompt}"
    for model_name, model in models.items():
        modified_text = full_text.replace("<mask>", "").replace(".", "").strip() if model_name == "papuGaPT2" else full_text           
        preds = model(modified_text, top_k=1)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"{pred['generated_text']}\n")
            else:
                print(f"{pred['sequence']}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Prompt: Zabarwienie emocjonalne wypowiedzi jest <mask>.

Predictions of HerBERT:
Film był tak nudny , że zsnęłam w kinie . Zabarwienie emocjonalne wypowiedzi jest słabe .


Predictions of RoBERTa:
Film był tak nudny, że zsnęłam w kinie. Zabarwienie emocjonalne wypowiedzi jest ogromne .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był tak nudny, że zsnęłam w kinie
Zabarwienie emocjonalne wypowiedzi jest bardzo ważne, ale nie jest to jedyny czynnik, który wpływa na odbiór filmu.
Nie wiem, czy to kwestia tego, że nie lubię tego typu filmów


Predictions of HerBERT:
Nie wiem , skąd tyle pozytywnych recenzji , moim zdaniem film był całkowicie schematyczny . Zabarwienie emocjonalne wypowiedzi jest słabe .


Predictions of RoBERTa:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny. Zabarwienie emocjonalne wypowiedzi jest ogromne .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny
Zabarwienie emocjonalne wypowiedzi jest bardzo dobre, ale nie jest to film, który można by oglądać bez końca.
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film


Predictions of HerBERT:
Film trwał ponad 3 godziny , ale nie żałuję żadnej minuty . Zabarwienie emocjonalne wypowiedzi jest świetne .


Predictions of RoBERTa:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty. Zabarwienie emocjonalne wypowiedzi jest ogromne .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty
Zabarwienie emocjonalne wypowiedzi jest bardzo ważne, ale nie jest to jedyny czynnik, który wpływa na odbiór filmu.
W filmie nie ma ani jednego momentu, w którym można by się uśmiechnąć


Predictions of HerBERT:
Film był reklamowany jako nowatorski , a okzał się odgrzewanym kotletem . Zabarwienie emocjonalne wypowiedzi jest ciekawe .


Predictions of RoBERTa:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem. Zabarwienie emocjonalne wypowiedzi jest nieznane .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem
Zabarwienie emocjonalne wypowiedzi jest bardzo ważne, ale nie jest to jedyny czynnik, który wpływa na odbiór filmu.
W filmie nie ma ani jednego słowa, które


Predictions of HerBERT:
Film długo nabierał rozpędu , ale gdy już go nabrał to nie dało się od niego oderwać . Zabarwienie emocjonalne wypowiedzi jest mocne .


Predictions of RoBERTa:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać. Zabarwienie emocjonalne wypowiedzi jest .


Predictions of papuGaPT2:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać
Zabarwienie emocjonalne wypowiedzi jest bardzo ważne, ale nie jest to jedyny czynnik, który wpływa na jej odbiór.
Nie wiem czy to kwestia tego



Dla promptu *Zabarwienie emocjonalne wypowiedzi jest <mask>* modele nie oceniły wypowiedzi w skali pozytywna-negatywna, a raczej mocna-słaba, pojawiał się też odpowiedzi *ciekawe*, *ważne*, jednak odpowiedzi te nie wydają się uzasadnione terścią opinii.

In [53]:
prompt = prompts[1]
print(f"\nPrompt: {prompt}")
for review in reviews:
    full_text = f"{review}\n{prompt}"
    for model_name, model in models.items():
        modified_text = full_text.replace("<mask>", "").replace(".", "").strip() if model_name == "papuGaPT2" else full_text           
        preds = model(modified_text, top_k=1)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"{pred['generated_text']}\n")
            else:
                print(f"{pred['sequence']}\n")


Prompt: Wypowiedź ma zdecydowanie wydźwięk <mask>.

Predictions of HerBERT:
Film był tak nudny , że zsnęłam w kinie . Wypowiedź ma zdecydowanie wydźwięk negatywny .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of RoBERTa:
Film był tak nudny, że zsnęłam w kinie. Wypowiedź ma zdecydowanie wydźwięk polityczny .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był tak nudny, że zsnęłam w kinie
Wypowiedź ma zdecydowanie wydźwięk polityczny. Nie wiem, czy to kwestia tego, że film jest o czymś, czy może tego, że jest to film o czymś.
Nie wiem,


Predictions of HerBERT:
Nie wiem , skąd tyle pozytywnych recenzji , moim zdaniem film był całkowicie schematyczny . Wypowiedź ma zdecydowanie wydźwięk negatywny .


Predictions of RoBERTa:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny. Wypowiedź ma zdecydowanie wydźwięk polityczny .



...


Predictions of papuGaPT2:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny
Wypowiedź ma zdecydowanie wydźwięk negatywny, ale nie jest to zarzut. Film jest bardzo dobry, ale nie jest to film, który można by oglądać bez końca.
Nie


Predictions of HerBERT:
Film trwał ponad 3 godziny , ale nie żałuję żadnej minuty . Wypowiedź ma zdecydowanie wydźwięk pozytywny .


Predictions of RoBERTa:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty. Wypowiedź ma zdecydowanie wydźwięk polityczny .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty
Wypowiedź ma zdecydowanie wydźwięk polityczny. Nie jest to jednak krytyka, a raczej krytyka, która ma na celu pokazanie, że nie jest to tylko krytyka, ale również krytyka, która ma na


Predictions of HerBERT:
Film był reklamowany jako nowatorski , a okzał się odgrzewanym kotletem . Wypowiedź ma zdecydowanie wydźwięk polityczny .


Predictions of RoBERTa:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem. Wypowiedź ma zdecydowanie wydźwięk polityczny .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem
Wypowiedź ma zdecydowanie wydźwięk polityczny. Nie jest to jednak krytyka, a raczej krytyka, a raczej krytyka, bo to, co się dzieje w Polsce, jest


Predictions of HerBERT:
Film długo nabierał rozpędu , ale gdy już go nabrał to nie dało się od niego oderwać . Wypowiedź ma zdecydowanie wydźwięk pozytywny .


Predictions of RoBERTa:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać. Wypowiedź ma zdecydowanie wydźwięk polityczny .


Predictions of papuGaPT2:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać
Wypowiedź ma zdecydowanie wydźwięk pozytywny, ale nie jest to do końca prawda. Nie wiem czy to kwestia tego, że nie jestem w stanie zrozumieć



Dla promptu *Wypowiedź ma zdecydowanie wydźwięk <mask>* modele udzieliły bardziej sensownych odpowiedzi. Najlepiej z określaniem sentymentu poradził sobie model HerBERT. Błędnej odpowiedzi udzielił jedynie w przypadku wypowiedzi, w której pojawiło się określenie *odgrzewany kotlet*. Wypowiedź ta miała na celu sprawdzenie, jak modele radzą sobie z rozumieniem przenośni. Okazało się, że w tym przypadku wszystkie modele określiły wydźwięk jako *polityczny*. Co ciekawe, to określenie pojawiało się zadziwiająco często, szczególnie w przypadku modelu RoBERTa.

In [54]:
prompt = prompts[2]
print(f"\nPrompt: {prompt}")
for review in reviews:
    full_text = f"{review}\n{prompt}"
    for model_name, model in models.items():
        modified_text = full_text.replace("<mask>", "").replace(".", "").strip() if model_name == "papuGaPT2" else full_text           
        preds = model(modified_text, top_k=1)
        print(f"\nPredictions of {model_name}:")
        for pred in preds:
            if model_name == "papuGaPT2":
                print(f"{pred['generated_text']}\n")
            else:
                print(f"{pred['sequence']}\n")


Prompt: Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest <mask>.

Predictions of HerBERT:
Film był tak nudny , że zsnęłam w kinie . Biorąc pod uwagę sentyment wypowiedzi , można stwierdzić , że opinia jest pozytywna .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of RoBERTa:
Film był tak nudny, że zsnęłam w kinie. Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest pozytywna .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był tak nudny, że zsnęłam w kinie
Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest bardzo dobra. Film jest bardzo dobry, ale nie jest to film, który można oglądać w kółko.
Nie wiem, czy


Predictions of HerBERT:
Nie wiem , skąd tyle pozytywnych recenzji , moim zdaniem film był całkowicie schematyczny . Biorąc pod uwagę sentyment wypowiedzi , można stwierdzić , że opinia jest pozytywna .


Predictions of RoBERTa:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny. Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest pozytywna .



...


Predictions of papuGaPT2:
Nie wiem, skąd tyle pozytywnych recenzji, moim zdaniem film był całkowicie schematyczny
Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest bardzo dobra. Film jest bardzo dobry, ale nie jest to film, który można oglądać na okrągło.



Predictions of HerBERT:
Film trwał ponad 3 godziny , ale nie żałuję żadnej minuty . Biorąc pod uwagę sentyment wypowiedzi , można stwierdzić , że opinia jest pozytywna .


Predictions of RoBERTa:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty. Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest pozytywna .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film trwał ponad 3 godziny, ale nie żałuję żadnej minuty
Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest bardzo pozytywna. Nie jest to jednak opinia, która ma na celu jedynie wywyższanie się, ale raczej nakłanianie do refleksji


Predictions of HerBERT:
Film był reklamowany jako nowatorski , a okzał się odgrzewanym kotletem . Biorąc pod uwagę sentyment wypowiedzi , można stwierdzić , że opinia jest pozytywna .


Predictions of RoBERTa:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem. Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest pozytywna .



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Predictions of papuGaPT2:
Film był reklamowany jako nowatorski, a okzał się odgrzewanym kotletem
Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest bardzo dobra. Nie ma co się dziwić, że film jest tak dobry.
Nie wiem czy to


Predictions of HerBERT:
Film długo nabierał rozpędu , ale gdy już go nabrał to nie dało się od niego oderwać . Biorąc pod uwagę sentyment wypowiedzi , można stwierdzić , że opinia jest pozytywna .


Predictions of RoBERTa:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać. Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest zgodna .


Predictions of papuGaPT2:
Film długo nabierał rozpędu, ale gdy już go nabrał to nie dało się od niego oderwać
Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest bardzo pozytywna. Nie jest to jednak opinia, która ma na celu jedynie wybić się



Dla promptu *Biorąc pod uwagę sentyment wypowiedzi, można stwierdzić, że opinia jest* modele określiły wszystkie wypowiedzi jako pozytywne lub bardzo dobre. 

Jak widać z powyżaszych przykładów konstrukcja promptu ma duży wpływ na odpowiedzi modeli.

### Answer the following questions:
   1. Which of the models produced the best results?
      - Modele papuGaPT2 i HerBERT dawały zdecydowanie lepsze rezultaty niż RoBERTa. Model papuGaPT2 okazał się lepszy w teście wiedzy, a model HerBERT lepiej poradził sobie z analizą sentymentu, choć w obu przypadkach odpowiedzi modeli były daleki od ideału. 

   2. Was any of the models able to capture Polish grammar?
      - Modele dość dobrze radziły sobie z zagadnieniami gramtycznymi. Problemy pojawiały się tylko przy wołaczu, co można tłumazyć tym, że jest to stosunkowo rzadko używany przypadek.
      
   3. Was any of the models able to capture long-distant relationships between the words?
      - Wszystkie modele poradziły sobie z tym zadaniem. Dla przeprowadzonego eksperymentu stosowały czasowniki w odpowiedniej formie. Jedynie model RoBERTa w niektórych przypadkach nie uzupełniał maski.

   4. Was any of the models able to capture world knowledge?
      - Modele generalnie słabo poradziły sobie z testem wiedzy. W tym wypadku najlepiej poradził sobie model papuGaPT2.
   5. Was any of the models good at doing zero-shot classification?
      - W klasyfikacji zero-shot najlepszy okazał się model HerBERT, jednak jego odpowiedzi były dalekie od ideału i mocno zależne od sformułowania promptu. 
      
   6. What are the most striking errors made by the models?
      -  Modele potrafią halucynować i udzielać odpowiedzi zupełnie niezgodnych z rzeczywistością lub niezwiązanych z pytaniem. Szczególnie widoczne było to w zadaniu dotyczącym wiedzy, w którym modele podawały nieprawdziwe informacje oraz w zadaniu dotyczącym rozpoznawania zabarwienia emocjonalnego wypowiedzi, w którym modele sotosowały określenia zupełnie niepasujące do analizy sentymentu.